## Gradient Descent Factorization

**Data parsing**

Initially, we retrieve and parse the data concerning both ratings, movies and users.

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics import mean_squared_error

**Data parsing**

Parse the data and retrieve the ratings

In [ ]:
#Import the Data Frame containing the ratings.
data = np.genfromtxt('./ratings.dat',
                     delimiter='::', usecols = (0, 1, 2), dtype = int)

#Convert to a data frame and rename the columns
ratings = pd.DataFrame(data)
ratings.columns = ['user', 'movie', 'rating']

In [ ]:
#Generate the training userxmovie matrix
rating_mat = ratings.pivot(index='user', columns='movie', values='rating')

In [ ]:
#We will work with numpy arrays since faster. 
ratings_np = np.matrix(ratings)
rating_mat_np = np.array(rating_mat)

#ratings_np represents the data frame of ratings with information encoded as <userID, moviesID, rating>. 
#rating_mat_np is the sparse matrix of ratings we use for the optimization algorithm.

**Implement the Gradient Descent method**

We implement the gradient descent method described by Tikk et al. in the paper named "On the Gravity Recommendation System". The procedure is based on the approximation of the sparse utility matrix by mean of two lower rank matrices (U and M) which are optimized towards the implementation of the so-called gravity recommendation system. We choose to initialize the matrices U and M such that their dot product yields a matrix with as elements the mean of all values of the utility matrix we want to approximate, subjected to random pertubations. U and V are dimensional reductions of the of the utility matrix representing every user and item as vectors of size $K$.

Given an $m*n$ data matrix $X$, the system starts from the hypothesis that the rating entry $X_{ij}$ is well approximated by the dot product of the vectors $u_i$ and $m_j$, representing respectively the $i^{th}$ row and $j^{th}$ column of $U$ and $M$. Therefore, our prediction for the matrix $X$ is represented by the matrix product of $U$ and $M$.

The algorithm will iterate across all available ratings from our dataset and optimize the value of the weights depending on the gradient of the error measure which, in our case, is represented by the squared difference between the observed and the predicted rating values. At each step, the value of the weights for a certain user and item will be updated with the gradient of the error function downsized by a learning rate positive constant. Ideally, the algroithm would cycle until convergence. For time necessities, we will limit our number of iterations to a maximum of 75.

In [ ]:
def initialize(ratings, n_users, n_movies, K):
    '''
    Given a rating data frame "ratings", the number of users and the number of movies and a constant K, the algorithm 
    initializes two matrices U and M of dimensions n_users*K and K*n_movies
    '''
    nan_mean = np.mean(ratings['rating'])
    initialize = np.sqrt(nan_mean/K)
    U = (np.repeat(initialize, K*n_users) + np.random.normal(0,1,K*n_users)).reshape((n_users, K))
    M = (np.repeat(initialize, K*n_movies) + np.random.normal(0,1,K*n_movies)).reshape((K, n_movies))
    return U,M

def RMSE(ratings, X, UM):
    '''
    RMSE computes the root mean squared error of two matrices, X and UM. Ratings is an array containing the list of
    ratings recorded as <userID, itemID, rating>.
    '''
    diff = X - UM
    n_non_missing = len(ratings)
    rmse = np.sqrt(np.nansum(diff**2)/n_non_missing)
    return rmse

def GRS(ratings, X, U, M,  row_names, col_names, iter = 75, lamb = 0.05, lrate = 0.005):
    '''
    Given an array of the form <userID, movieID, ratings>, a sparse matrix X and the two initialized matrices U and V, 
    implement the gravity recommendation system algorithm. The dictionaries row_names and col_names encode for the mapping
    between the ID of a user or item and the row/column index of the matrix X.
    '''
    #In res, we append the result of the RMSE error for each iteration for later plotting.
    res = []
    #Compute the first RMSE. 
    RMSE_new = RMSE(ratings, X, U.dot(M))
    #Initialize the RMSE_old to infinity.
    RMSE_old = np.inf
    count = 1
    #The algorithm will stop after a default number of iterations or when the old and new RMSEs converge. 
    while count <= iter and RMSE_old != RMSE_new:
        for rat in ratings:
            (i,j) = row_names[rat[0,0]], col_names[rat[0,1]]
            e = X[i,j] - U[i,:].dot(M[:,j])
            U_copy = copy.copy(U[i,:]) #Copy the object U to avoid side effects as suggested in th Simon Funk's blog.
            U[i,:] += lrate*(2*e*M[:,j] - lamb*U[i,:])
            M[:,j] += lrate*(2*e*U_copy - lamb*M[:,j])
        RMSE_old = RMSE_new
        RMSE_new = RMSE(ratings, X, U.dot(M))
        res.append(RMSE_new)
        count += 1
    return U.dot(M), res

In the following script, we will estimate the test error of our approach via 5-fold cross-validation. Here are the parameters we used for producing the results:
* The regularization factor $\lambda$ is set equal to 0.05
* The learning rate is set to 0.005
* The value of K (hence, the number of features for each user and item) is equal to 10

**Test by cross-validation**

In [ ]:
#Implement 5-fold cross validation

#We will keep a partial error for plotting
partial_error = []

#The parameters 
K = 10
iter = 75
lamb = 0.05
lrate = 0.005

#Set a seed for reproducibility
np.random.seed(888)

#Generate the 5 folds for all rows of the utility matrix of input.
nfolds = 5
folds = np.array([x%nfolds for x in range(len(ratings))])
np.random.shuffle(folds)


#Set up the vectors of errors.
errors_5folds_rmse_training = []
errors_5folds_rmse_test = [] 

'''
We will cycle through the possible folds (from 1 to 5) and leave one set of observations (test set) out of the training process.
Then, we use it to perform predictions and compute the error. 
'''

for i in range(5):
    print('Start iteration '+ str(i))
    #Fix the rows of the ratings Data Frame that will be the training set.
    train = ratings.loc[folds!=i, :]
    train_mat = train.pivot(index='user', columns='movie', values='rating')
    train_np = np.matrix(train)
    train_mat_np = np.array(train_mat)
    
    #Create lookup dictionary associating the indices of the numpy array to the
    #respective movieID and userID.
    col_names_train = {train_mat.columns[j]:j for j in range(len(train_mat.columns))}
    row_names_train = {train_mat.index[j]:j for j in range(len(train_mat.index))} 
    
    #Initialize U and M and run the prediction.
    U, M = initialize(train, np.size(train_mat_np,0), np.size(train_mat_np,1), K)
    pred, res = GRS(train_np, train_mat_np, U, M, row_names_train, col_names_train)
    
    #The global mean of the training matrix will be used as fallback rule movies or user ID's absent in the training set.
    global_mean = np.mean(train['rating'])
    
    #Initialize test set
    test =  ratings.loc[folds == i, :]
    test_np = np.matrix(test)
    test_pred = []
    
    #Create the prediction vector of the test set. If a user or item are not present
    #in the training set, predict their rating as the gloal mean if the prediction matrix.
    for val in test_np:
        if val[0,0] in row_names_train and val[0,1] in col_names_train:
            test_pred.append(pred[row_names_train[val[0,0]], col_names_train[val[0,1]]])
        else:
            test_pred.append(global_mean)
    test_pred = np.array(test_pred)
    test_obs = np.array(test['rating'])
    
    #Extra fallback rules to improve performance
    test_pred = np.where(test_pred > 5, 5, test_pred)
    test_pred = np.where(test_pred < 1, 1, test_pred)
    
    #Calculate and store the errors on the training set and test set.
    errors_5folds_rmse_training.append(RMSE(train_np,train_mat_np,pred))
    
    #Calculate the error on the test set and store it.
    test_RMSE = np.sqrt(np.sum((test_pred-test_obs)**2)/len(test_pred)) 
    errors_5folds_rmse_test.append(test_RMSE)
    partial_error.append(res)   
    
    print('End of iteration', i)    

We now analyse our results computing the estimated runtime and the mean RMSE over the training set. We will also plot the convergence of the RMSE on the training set across the different folds.

In [ ]:
#Average across the folds to compute the errors. 
cv_error_rmse_train = np.mean(errors_5folds_rmse_training)
cv_error_rmse_test = np.mean(errors_5folds_rmse_test)

print('The average rmse over the training set is: ', cv_error_rmse_train)
print('The average rmse over the test set is: ', cv_error_rmse_test)